In [1]:
import numpy as np
import cv2

In [51]:
X_train=np.load('npy data/train/images.npy')
trainy=np.load('npy data/train/labels.npy')
X_test=np.load('npy data/test/images.npy')
testy=np.load('npy data/test/labels.npy')

In [52]:
trainx=list()
Y_train=list()
testx=list()
Y_test=list()
for i in X_train:
    i=cv2.GaussianBlur(i,(5,5),cv2.BORDER_DEFAULT)
    trainx.append(cv2.cvtColor(i,cv2.COLOR_BGR2GRAY))
for i in X_test:
    i=cv2.GaussianBlur(i,(5,5),cv2.BORDER_DEFAULT)
    trainx.append(cv2.cvtColor(i,cv2.COLOR_BGR2GRAY))
for i in trainy:
    Y_train.append(i)
for i in testy:
    Y_train.append(i)
X_train=np.array(trainx)
trainy=np.array(Y_train)
X_test=np.array(testx)
testy=np.array(Y_test)

In [53]:
X_train=X_train/255
X_test=X_test/255

In [5]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,trainy,testy=train_test_split(X_train,trainy,test_size=0.2,random_state=420)

In [25]:
from keras.utils import np_utils,to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.applications import VGG16
import keras.backend as K

In [8]:
X_train.shape

(4672, 150, 150)

In [9]:
compute_class_weight(class_weight='balanced',classes=[0,1],y=trainy)

array([1.88387097, 0.68065268])

In [10]:
Y_train=to_categorical(trainy,2)
Y_test=to_categorical(testy,2)

In [11]:
X_train=X_train.reshape(-1,150,150,1)
X_test=X_test.reshape(-1,150,150,1)

In [32]:
def my_filter(shape, dtype=None):

    f = np.array([
            [[[1]], [[-1]], [[0]]],
            [[[-1]], [[6]], [[-1]]],
            [[[0]], [[-1]], [[0]]]
        ])
#     print(shape)
    assert f.shape == shape
    return K.variable(f, dtype='float32')

In [33]:
model=Sequential()
model.add(Conv2D(filters=1,kernel_size=3,kernel_initializer=my_filter,padding='same',activation='relu',input_shape=(150,150,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 150, 150, 1)       10        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 1)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 75, 75, 32)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 37, 37, 64)        8256      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 18, 64)      

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=50,epochs=20,verbose=1)

Epoch 1/20
94/94 [==============================] - 28s 294ms/step - loss: 1.8627 - accuracy: 0.6764 - val_loss: 0.1882 - val_accuracy: 0.9298
Epoch 2/20
94/94 [==============================] - 27s 289ms/step - loss: 0.1850 - accuracy: 0.9270 - val_loss: 0.1617 - val_accuracy: 0.9341
Epoch 3/20
94/94 [==============================] - 29s 304ms/step - loss: 0.1567 - accuracy: 0.9382 - val_loss: 0.1646 - val_accuracy: 0.9443
Epoch 4/20
94/94 [==============================] - 27s 292ms/step - loss: 0.1476 - accuracy: 0.9414 - val_loss: 0.1507 - val_accuracy: 0.9409
Epoch 5/20
94/94 [==============================] - 26s 282ms/step - loss: 0.1269 - accuracy: 0.9532 - val_loss: 0.1442 - val_accuracy: 0.9443
Epoch 6/20
94/94 [==============================] - 26s 282ms/step - loss: 0.1308 - accuracy: 0.9512 - val_loss: 0.1403 - val_accuracy: 0.9461
Epoch 7/20
94/94 [==============================] - 27s 282ms/step - loss: 0.1024 - accuracy: 0.9609 - val_loss: 0.1468 - val_accuracy: 0.9409

In [36]:
acc=model.evaluate(X_test,Y_test,verbose=0)[1]
print('accuracy = '+str(acc*100))

accuracy = 94.09246444702148


In [37]:
pred=model.predict_classes(X_test)

c:\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [38]:
print(confusion_matrix(pred,testy))
print(classification_report(pred,testy))

[[315  49]
 [ 20 784]]
              precision    recall  f1-score   support

           0       0.94      0.87      0.90       364
           1       0.94      0.98      0.96       804

    accuracy                           0.94      1168
   macro avg       0.94      0.92      0.93      1168
weighted avg       0.94      0.94      0.94      1168



In [39]:
from keras.models import save_model,load_model

In [40]:
model.save('models/model4.h5')

In [54]:
val_images=list()
val=np.load('npy data/validation/images.npy')
for i in val:
    i=cv2.GaussianBlur(i,(5,5),cv2.BORDER_DEFAULT)
    val_images.append(cv2.cvtColor(i,cv2.COLOR_BGR2GRAY))
val_labels=np.load('npy data/validation/labels.npy')
val=np.array(val_images)
val=val/255
val=val.reshape(-1,150,150,1)

In [55]:
example=np.array([val[1]])

In [56]:
model.predict_classes(val)

c:\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [44]:
example.shape

(1, 150, 150, 1)

In [41]:
val_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [195]:
im=cv2.imread("./download/normal/5.jpg")

In [84]:
validation_images=np.load('npy data/validation2/images.npy')
validation_labels=np.load('npy data/validation2/labels.npy')
validation_labels_categorical=to_categorical(validation_labels,2)

In [81]:
pred=model.predict_classes(validation_images)

c:\python\python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [85]:
acc=model.evaluate(validation_images,validation_labels_categorical,verbose=0)[1]
print(acc)
print(confusion_matrix(pred,validation_labels))
print(classification_report(pred,validation_labels))

0.9356215596199036
[[1516  310]
 [  67 3963]]
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      1826
           1       0.93      0.98      0.95      4030

    accuracy                           0.94      5856
   macro avg       0.94      0.91      0.92      5856
weighted avg       0.94      0.94      0.93      5856



In [196]:
im=cv2.resize(im,(150,150),interpolation = cv2.INTER_NEAREST)
im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
im=im.reshape(-1,150,150,1)
im=im/255

In [197]:
model.predict_classes(im)

array([1], dtype=int64)

In [108]:
np.array([X_train]).shape

(1, 5840, 150, 150)

In [8]:
model=load_model('finilized/gray(150,150)/b&w(150,150).h5')

In [75]:
im=cv2.imread("D:/Sairam Gundala/academics/kaggle/pneumonia/All/540270208.jpeg")
im=cv2.GaussianBlur(im,(5,5),cv2.BORDER_DEFAULT)
im=cv2.resize(im,(150,150),interpolation = cv2.INTER_NEAREST)
im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
im=im.reshape(-1,150,150,1)
im=im/255

In [76]:
model.predict_classes(im)

array([0], dtype=int64)